# Modify Background Database

In this project, certain processes based on exisiting processes contained in the background databases must be added to their respective background database. These processes are based on:

- diesel, burned in building machine - GLO; from `EI_DB_NAME`;
- transport, freight, lorry 7.5-16 metric ton, EURO5 - RER; from `EI_DB_NAME`;
- transport, passenger car, medium size, petrol, EURO 5 - RER; from `EI_DB_NAME`;
- basalt quarry operation - RER; from `EI_DB_NAME`;
- gravel production, crushed - RoW; from `EI_DB_NAME`;
- cellulose fibre production - RoW from `EI_DB_NAME`.


In [1]:
import bw2data as bd
import bw2io as bi

In [3]:
#Importing the variables with the project name and background db
from project_details import EI_DB_NAME, PROJECT_NAME

In [4]:
#Set project
bd.projects.set_current(PROJECT_NAME)
#Check databases included in project
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

In [5]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert EI_DB_NAME in bd.databases

## Adding modified diesel processes
First, two processes based on the ecoinvent process "diesel, burned in building machine - GLO" must be added. These processes correspond to the diesel consumption attributed to the machinery use to lay and mill asphalt during the construction (A5) and EOL (C1) life cycle phases of our system under analysis. Therefore, we will create two processes named as follows:

- A5, diesel, burned in building machine - GLO
- C1, diesel, burned in building machine - GLO

The emissions of these processes are modified to comply with the specifications of the [NL-PCR](https://www.bouwendnederland.nl/media/12923/pcr-asfalt-v20.pdf).

The emissions that we are modifying and their respective new values are:

| Name                                               | A5 (kg)               | C1 (kg)               |
|----------------------------------------------------|-----------------------|-----------------------|
| Ammonia                                            | 2.33222e-07           | 2.33222e-07           |
| Carbon dioxide, fossil                            | 7.44086e-05           | 7.44086e-05           |
| Carbon monoxide, fossil                           | 3.19444e-08           | 1.00901e-08           |
| Dinitrogen monoxide                               | 4.99833e-10           | 4.99833e-10           |
| Nitrogen oxides                                   | 7.67056e-08           | 3.83334e-08           |
| NMVOC, non-methane volatile organic compounds     | 8.91294e-10           | 4.08334e-10           |
| Particulate Matter, < 2.5 um                      | 1.29832e-09           | 1.63390e-09           |
| Particulate Matter, > 10 um                       | 6.83056e-11           | 3.22335e-09           |
| Particulate Matter, > 2.5 um and < 10um           | 1.23587e-09           | 1.89703e-09           |
| Sulfur dioxide                                    | 4.66667e-10           | 4.66667e-10           |

Furthermore, the diesel exchange *market group for diesel - GLO* must be replaced by *market for diesel, low-sulfur - Europe without Switzerland* in both new processes.

In [8]:
#Search for original diesel process in background database
EI_DB = bd.Database(EI_DB_NAME)
EI_DB.search('diesel, burned in building machine')

['market for diesel, burned in building machine' (megajoule, GLO, None),
 'diesel, burned in building machine' (megajoule, GLO, None),
 'propane, burned in building machine' (megajoule, GLO, None),
 'market for propane, burned in building machine' (megajoule, GLO, None),
 'market for diesel, burned in diesel-electric generating set, 10MW' (megajoule, GLO, None)]

In [10]:
#Copy existing diesel process
diesel = EI_DB.search('diesel, burned in building machine')[1]
diesel_A5 = diesel.copy(name='A5, diesel, burned in building machine') 
diesel_C1 = diesel.copy(name='C1, diesel, burned in building machine') 

In [11]:
#Check that new processes are saved to background database
EI_DB = bd.Database(EI_DB_NAME)
EI_DB.search('diesel, burned in building machine')

['market for diesel, burned in building machine' (megajoule, GLO, None),
 'diesel, burned in building machine' (megajoule, GLO, None),
 'A5, diesel, burned in building machine' (megajoule, GLO, None),
 'C1, diesel, burned in building machine' (megajoule, GLO, None),
 'propane, burned in building machine' (megajoule, GLO, None),
 'market for propane, burned in building machine' (megajoule, GLO, None),
 'market for diesel, burned in diesel-electric generating set, 10MW' (megajoule, GLO, None)]

In [12]:
#Look at exhanges characterizing activity
for exc in diesel_A5.exchanges():
    print(exc)

Exchange: 1.0 megajoule 'A5, diesel, burned in building machine' (megajoule, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 1.34e-07 unit 'market for building machine' (unit, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 0.0234 kilogram 'market group for diesel' (kilogram, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 8.943599999999386e-05 kilogram 'market for lubricating oil' (kilogram, RER, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 0.00042456400000000623 kilogram 'market for lubricating oil' (kilogram, RoW, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: -0.0003694187171108785 kilogram 'market for waste mineral oil' (kilogram, RoW, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: -0.00014108849302253373 kilogram 'market for waste mineral oil' (kil

In [14]:
#detail look to exchanges of interest
[exc for exc in diesel_A5.exchanges()][8].as_dict() #Ammonia

{'flow': '87883a4e-1e3e-4c9d-90c0-f1bea36f8014',
 'type': 'biosphere',
 'name': 'Ammonia',
 'classifications': {'CPC': []},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'water content': {'amount': 0.0,
   'comment': 'water mass/dry mass',
   'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'unit': 'kilogram',
 'amount': 4.67e-07,
 'pedigree': {'reliability': 1,
  'completeness': 5,
  'temporal correlation': 5,
  'geographical correlation': 3,
  'further technological correlation': 1},
 'uncertainty type': 2,
 'loc': -14.576936579277515,
 'scale': 0.3132091952673165,
 'scale without pedigree': 0.22360679774997896

In [15]:
#Create dictionaries with new emissions for each activity
A5_emissions = [
    {"name": "Ammonia", "amount": 2.33222e-07},
    {"name": "Carbon dioxide, fossil", "amount": 7.44086e-05},
    {"name": "Carbon monoxide, fossil", "amount": 3.19444e-08},
    {"name": "Dinitrogen monoxide", "amount": 4.99833e-10},
    {"name": "Nitrogen oxides", "amount": 7.67056e-08},
    {"name": "NMVOC, non-methane volatile organic compounds", "amount": 8.91294e-10},
    {"name": "Particulate Matter, < 2.5 um", "amount": 1.29832e-09},
    {"name": "Particulate Matter, > 10 um", "amount": 6.83056e-11},
    {"name": "Particulate Matter, > 2.5 um and < 10um", "amount": 1.23587e-09},
    {"name": "Sulfur dioxide", "amount": 4.66667e-10}
]

C1_emissions = [
    {"name": "Ammonia", "amount": 2.33222e-07},
    {"name": "Carbon dioxide, fossil", "amount": 7.44086e-05},
    {"name": "Carbon monoxide, fossil", "amount": 1.00901e-08},
    {"name": "Dinitrogen monoxide", "amount": 4.99833e-10},
    {"name": "Nitrogen oxides", "amount": 3.83334e-08},
    {"name": "NMVOC, non-methane volatile organic compounds", "amount": 4.08334e-10},
    {"name": "Particulates, <2.5 µm", "amount": 1.63390e-09},
    {"name": "Particulates, >2.5 µm and <10 µm", "amount": 3.22335e-09},
    {"name": "Particulates, >10 µm", "amount": 1.89703e-09},
    {"name": "Sulphur dioxide", "amount": 4.66667e-10}
]

Now we modify the exchanges of the new activities

In [18]:
# Iterate over exchanges in diesel_A5
for exc in diesel_A5.exchanges():
    # Check if the exchange name is in A5_emissions
    for emission in A5_emissions:
        if exc['name'] == emission['name']:
            # Update the amount in the exchange
            exc['amount'] = emission['amount']
            exc.save()
            break  # Exit the loop once the exchange is found

In [20]:
#Iterate over exchanges in diesel_C1
for exc in diesel_C1.exchanges():
    # Check if the exchange name is in C1_emissions
    for emission in C1_emissions:
        if exc['name'] == emission['name']:
            # Update the amount in the exchange
            exc['amount'] = emission['amount']
            exc.save()
            break  # Exit the loop once the exchange is found

Verification: were the exchange amounts indeed updated?

In [19]:
#Look at exhanges characterizing activity diesel_A5
for exc in diesel_A5.exchanges():
    print(exc)

Exchange: 1.0 megajoule 'A5, diesel, burned in building machine' (megajoule, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 1.34e-07 unit 'market for building machine' (unit, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 0.0234 kilogram 'market group for diesel' (kilogram, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 8.943599999999386e-05 kilogram 'market for lubricating oil' (kilogram, RER, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 0.00042456400000000623 kilogram 'market for lubricating oil' (kilogram, RoW, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: -0.0003694187171108785 kilogram 'market for waste mineral oil' (kilogram, RoW, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: -0.00014108849302253373 kilogram 'market for waste mineral oil' (kil

Now we change the diesel exchange from *market group for diesel - GLO* to *market for diesel, low-sulfur - Europe without Switzerland*.

In [126]:
#Store original diesel exchange as a variable
diesel_og = [exc for exc in diesel_A5.exchanges()][2] #Market group for diesel
diesel_og

Exchange: 0.0234 kilogram 'market group for diesel' (kilogram, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>

In [132]:
#Take a closer look to diesel_og
diesel_og.as_dict()

{'flow': '291fc06d-1b3e-4077-aabb-346b588ed24b',
 'type': 'technosphere',
 'name': 'diesel',
 'classifications': {'CPC': ['33360: Gas oil']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.865, 'unit': 'kg'},
  'carbon content': {'amount': 0.865, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.865,
   'comment': 'C-content of diesel is 86.5% (ecoinvent v2.2 report 6_IV, Tab 3.14)',
   'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'heating value, net': {'amount': 42.8,
   'comment': 'Based on literature, page 16, Report6, part 4',
   'unit': 'MJ'},
  'water content': {'amount': 0.0,
   'comment': 'water mass/dry mass',
   'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': '4df5d193-687d-534a-b8e5-9f80e6a93b2d',
 'unit': 'kilogram',
 'comment': 'EcoSpold

In [113]:
#find replacement exchange for diesel_og: market for diesel, low-sulfur - Europe without Switzerland
EI_DB.search('diesel, low-sulfur')

['market for diesel, low-sulfur' (kilogram, Europe without Switzerland, None),
 'market for diesel, low-sulfur' (kilogram, RoW, None),
 'market for diesel, low-sulfur' (kilogram, CH, None),
 'diesel production, low-sulfur' (kilogram, CH, None),
 'market for diesel, low-sulfur' (kilogram, BR, None),
 'market for diesel, low-sulfur' (kilogram, IN, None),
 'market for diesel, low-sulfur' (kilogram, CO, None),
 'market for diesel, low-sulfur' (kilogram, ZA, None),
 'market for diesel, low-sulfur' (kilogram, PE, None),
 'diesel, low-sulfur, import from Europe' (kilogram, CH, None),
 'market group for diesel, low-sulfur' (kilogram, RER, None),
 'market group for diesel, low-sulfur' (kilogram, GLO, None),
 'diesel, low-sulfur, import from RoW' (kilogram, BR, None),
 'diesel, low-sulfur, import from RoW' (kilogram, IN, None),
 'diesel, low-sulfur, import from RoW' (kilogram, ZA, None),
 'diesel, low-sulfur, import from RoW' (kilogram, PE, None),
 'diesel, low-sulfur, import from RoW' (kilogram

In [134]:
#store replacement activity as variable
diesel_ls = EI_DB.search('diesel, low-sulfur')[0]

In [141]:
#Take a closer look to diesel_ls
diesel_ls.as_dict()

{'comment': "This is a market activity. Each market represents the consumption mix of a product in a given geography, connecting suppliers with consumers of the same product in the same geographical area. Markets group the producers and also the imports of the product (if relevant) within the same geographical area. They also account for transport to the consumer and for the losses during that process, when relevant.\nThis is the market for  'diesel, low-sulfur', in the geography of Europe without Switzerland.\nInventory for the distribution of petroleum product to the final consumer (household, car, power plant, etc.) including all necessary transports.\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinven

In [135]:
#add diesel_ls to diesel_x exchanges based on the diesel_og exchange parameters
diesel_ls_A5 = diesel_A5.new_exchange(input=diesel_ls.key, amount=diesel_og.amount, unit=diesel_og.unit, 
                                      type='technosphere')
diesel_ls_C1 = diesel_C1.new_exchange(input=diesel_ls.key, amount=diesel_og.amount, unit=diesel_og.unit, 
                                      type='technosphere')
diesel_ls_A5.save()
diesel_ls_C1.save()
diesel_A5.save()
diesel_C1.save()

In [136]:
#Verify characterizing activity diesel_A5
[exc for exc in diesel_A5.exchanges()][2].as_dict()

{'flow': '291fc06d-1b3e-4077-aabb-346b588ed24b',
 'type': 'technosphere',
 'name': 'diesel',
 'classifications': {'CPC': ['33360: Gas oil']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.865, 'unit': 'kg'},
  'carbon content': {'amount': 0.865, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.865,
   'comment': 'C-content of diesel is 86.5% (ecoinvent v2.2 report 6_IV, Tab 3.14)',
   'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'heating value, net': {'amount': 42.8,
   'comment': 'Based on literature, page 16, Report6, part 4',
   'unit': 'MJ'},
  'water content': {'amount': 0.0,
   'comment': 'water mass/dry mass',
   'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': '4df5d193-687d-534a-b8e5-9f80e6a93b2d',
 'unit': 'kilogram',
 'comment': 'EcoSpold

In [142]:
#Delete diesel exchange
[exc for exc in diesel_A5.exchanges()][2].delete() #Market group for diesel
[exc for exc in diesel_C1.exchanges()][2].delete() #Market group for diesel

In [143]:
#Verify characterizing activity diesel_A5
for exc in diesel_A5.exchanges():
    print(exc)

Exchange: 1.0 megajoule 'A5, diesel, burned in building machine' (megajoule, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 1.34e-07 unit 'market for building machine' (unit, GLO, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 8.943599999999386e-05 kilogram 'market for lubricating oil' (kilogram, RER, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: 0.00042456400000000623 kilogram 'market for lubricating oil' (kilogram, RoW, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: -0.0003694187171108785 kilogram 'market for waste mineral oil' (kilogram, RoW, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: -0.00014108849302253373 kilogram 'market for waste mineral oil' (kilogram, Europe without Switzerland, None) to 'A5, diesel, burned in building machine' (megajoule, GLO, None)>
Exchange: -3.4927898665877724e-06

In [144]:
#Verify diesel_ls exchange
[exc for exc in diesel_A5.exchanges()][27].as_dict()

{'output': ('ecoinvent-391-cutoff', '50aace6627f8438488f9c0c1ab58e8a7'),
 'input': ('ecoinvent-391-cutoff', '93406e2b0b71b9681c5aba419dddc642'),
 'amount': 0.0234,
 'unit': 'kilogram',
 'type': 'technosphere'}

## Adding modified transport processes

Two new transport processes to model the extra fuel consumption due to pavement deterioration in the use phase (B) must be added to the background. These processes are based on the following ecoinvent processes:

The new processes are modified to only include the exchanges related to fuel (diesel and petrol). The name of the new processes is:

- transport, freight, lorry 7.5-16 metric ton, EURO5 - RER
- transport, passenger car, medium size, petrol, EURO 5 - RER

The new processes are modified to only include the technosphere exchanges related to fuel (diesel and petrol). The names of the new processeses are:
- B, transport, freight, lorry 7.5-16 metric ton, EURO5 - RER
- B, transport, passenger car, medium size, petrol, EURO 5 - RER

In [26]:
#Search for transport processes in background database
EI_DB.search('transport, freight, lorry 7.5-16 metric ton, EURO5')

['transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, BR, None),
 'market for transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, BR, None),
 'market for transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None),
 'market for transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None),
 'transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RER, None),
 'transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RoW, None),
 'market for transport, freight, lorry with refrigeration machine, 7.5-16 ton, EURO5, R134a refrigerant, cooling' (ton kilometer, GLO, None),
 'market for transport, freight, lorry with refrigeration machine, 7.5-16 ton, EURO5, R134a refrigerant, freezing' (ton kilometer, GLO, None),
 'transport, freight, lorry 3.5-7.

In [27]:
EI_DB.search('transport, passenger car, medium size, petrol, EURO 5')

['transport, passenger car, medium size, petrol, EURO 5' (kilometer, RoW, None),
 'transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None),
 'market for transport, passenger car, medium size, petrol, EURO 5' (kilometer, GLO, None),
 'transport, passenger car, small size, petrol, EURO 5' (kilometer, RER, None),
 'transport, passenger car, large size, petrol, EURO 5' (kilometer, RER, None),
 'transport, passenger car, small size, petrol, EURO 5' (kilometer, RoW, None),
 'transport, passenger car, large size, petrol, EURO 5' (kilometer, RoW, None),
 'transport, passenger car, medium size, diesel, EURO 5' (kilometer, RoW, None),
 'transport, passenger car, medium size, diesel, EURO 5' (kilometer, RER, None),
 'transport, passenger car, medium size, natural gas, EURO 5' (kilometer, RER, None),
 'transport, passenger car, medium size, natural gas, EURO 5' (kilometer, RoW, None),
 'transport, passenger car, small size, diesel, EURO 5' (kilometer, RER, None),
 'transport,

In [55]:
#Copy existing lorry process
lorry = EI_DB.search('transport, freight, lorry 7.5-16 metric ton, EURO5')[4]
lorry_B = lorry.copy(name='B, transport, freight, lorry 7.5-16 metric ton, EURO5') 

In [ ]:
#Copy existing car process
car = EI_DB.search('transport, passenger car, medium size, petrol, EURO 5')[1]
car_B = car.copy(name='B, transport, passenger car, medium size, petrol, EURO 5') 

In [56]:
#Verify that the new transport processes were recorded
EI_DB.search('B, transport,')

['B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None),
 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None),
 'market for cement, CEM III/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM III/B' (kilogram, CH, None),
 'market for cement, CEM II/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM IV/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM V/B' (kilogram, Europe without Switzerland, None),
 'market for cement, CEM V/B' (kilogram, CH, None),
 'market for cement, CEM II/B' (kilogram, CH, None),
 'market for cement, CEM IV/B' (kilogram, CH, None),
 'market for cement, CEM III/B' (kilogram, RoW, None),
 'market for cement, CEM V/B' (kilogram, RoW, None),
 'market for cement, CEM IV/B' (kilogram, RoW, None),
 'market for cement, CEM II/B' (kilogram, RoW, None),
 'market for cement, CEM II/B-S' (kilogram, ZA, None),
 'market for cement, CEM II/B-L' (kilogr

First, let's update B, transport, freight, lorry 7.5-16 metric ton, EURO5

In [57]:
#Look at technosphere exhanges characterizing activity lorry_B
for exc in lorry_B.technosphere():
    print(exc)

Exchange: -2.29493343465046e-05 kilogram 'market for brake wear emissions, lorry' (kilogram, GLO, None) to 'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)>
Exchange: 0.047302103 kilogram 'market group for diesel, low-sulfur' (kilogram, RER, None) to 'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)>
Exchange: 5.63e-07 unit 'market for lorry, 16 metric ton' (unit, RER, None) to 'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)>
Exchange: 5.63e-07 unit 'market for maintenance, lorry 16 metric ton' (unit, RER, None) to 'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)>
Exchange: 0.00109 meter-year 'market for road' (meter-year, GLO, None) to 'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)>
Exchange: 0.000392949198718733 meter-year 'market for road maintenance' (meter-year, RER, None) to 'B, transport, freight, lorry 7.5-16

In [58]:
#detail look to exchanges of interest
[exc for exc in lorry_B.technosphere()][1].as_dict() #market group for diesel, low-sulfur

{'flow': 'b80eb2d5-1256-41bb-a61d-83252d0dfd4d',
 'type': 'technosphere',
 'name': 'diesel, low-sulfur',
 'classifications': {'CPC': ['33360: Gas oil']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.865, 'unit': 'kg'},
  'carbon content': {'amount': 0.865, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.865,
   'comment': 'C-content of diesel low-sulfur is 86.5% (ecoinvent v2.2 report 6_IV, Tab 3.14)',
   'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'heating value, net': {'amount': 42.8,
   'comment': 'Based on literature, page 16, Report6, part 4',
   'unit': 'MJ'},
  'water content': {'amount': 0.0,
   'comment': 'water mass/dry mass',
   'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': '427d868d-e90c-5da5-bde4-ff61ebd30a6f',
 'unit': 'kilogram'

In [59]:
# Iterate over exchanges in lorry_B
for exc in lorry_B.technosphere():
        if exc['name'] != 'diesel, low-sulfur':
            # Delete exchange
            exc.delete()
            exc.save()

Let's verify the exchanges for the activity  now

In [60]:
#Look at exhanges characterizing activity lorry_B
for exc in lorry_B.technosphere():
    print(exc)

Exchange: 0.047302103 kilogram 'market group for diesel, low-sulfur' (kilogram, RER, None) to 'B, transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None)>


Let's repeat for B, transport, passenger car, medium size, petrol, EURO 5

In [49]:
#Look at technosphere exhanges characterizing activity car_B
for exc in car_B.technosphere():
    print(exc)

Exchange: -7.55254e-06 kilogram 'market for brake wear emissions, passenger car' (kilogram, GLO, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>
Exchange: 8.6021505376344e-06 unit 'maintenance, passenger car' (unit, RER, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>
Exchange: 0.0106666666666667 kilogram 'market for passenger car, petrol/natural gas' (kilogram, GLO, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>
Exchange: 0.06090593821630414 kilogram 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>
Exchange: 0.001160855383695865 kilogram 'market for petrol, low-sulfur' (kilogram, CH, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>
Exchange: 0.0009113964 meter-year 'market for road' (meter-year, GLO,

In [50]:
#detail look to exchanges of interest
[exc for exc in car_B.technosphere()][3].as_dict() #market for petrol, low-sulfur - Europe without Szwitzerland
[exc for exc in car_B.technosphere()][4].as_dict() #market for petrol, low-sulfur - CH

{'flow': '894c4486-e821-4d6c-b99f-7e3f63d4a083',
 'type': 'technosphere',
 'name': 'petrol, low-sulfur',
 'classifications': {'CPC': ['333: Petroleum oils and oils obtained from bituminous materials, other than crude; preparations n.e.c. containing by […]']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.865, 'unit': 'kg'},
  'carbon content': {'amount': 0.865, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.865, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'heating value, net': {'amount': 42.5,
   'comment': 'Based on literature, page 611, Report 17: Jungbluth, N., Chudacoff, M., Dauriat, A., Dinkel, F., Doka, G., Faist Emmenegger, M., Gnansounou, E., Kljun, N., Schleiss, K., Spielmann, M., Stettler, C., Sutter, J. 2007: Life Cycle Inventories of Bioenergy. ecoinvent report No. 17, Swiss Centre for Life Cycle Inventories, Dübendorf, CH

In [51]:
# Iterate over exchanges in car_B
for exc in car_B.technosphere():
        if exc['name'] != 'petrol, low-sulfur':
            #delete exchange
            exc.delete()
            exc.save()

In [52]:
#Look at exhanges characterizing activity car_B
for exc in car_B.technosphere():
    print(exc)

Exchange: 0.06090593821630414 kilogram 'market for petrol, low-sulfur' (kilogram, Europe without Switzerland, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>
Exchange: 0.001160855383695865 kilogram 'market for petrol, low-sulfur' (kilogram, CH, None) to 'B, transport, passenger car, medium size, petrol, EURO 5' (kilometer, RER, None)>


## Adding basalt quarry operation modified process

This processs is based on the following ecoinvent process:

- basalt quarry operation - RER

The new process is modified to set the basalt biosphere exchange amount to 0, the technosphere limestone quarry infrastructure exchange amount to 0, and multiply all the rest of the technosphere exchanges by 1,04. 

The name of the new process is set to

- basalt quarry operation (for crushed stone) - RER

In [61]:
#Search for basalt quarry operation in background database
EI_DB.search('basalt quarry operation')

['basalt quarry operation' (kilogram, RoW, None),
 'basalt quarry operation' (kilogram, RER, None)]

In [63]:
#Copy existing basalt process
basalt = EI_DB.search('basalt quarry operation')[1]
basalt_cs = basalt.copy(name='basalt quarry operation (for crushed stone)') 

In [65]:
#Verify that the new basalt process was recorded
EI_DB.search('basalt quarry operation')

['basalt quarry operation' (kilogram, RoW, None),
 'basalt quarry operation' (kilogram, RER, None),
 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)]

In [67]:
#Look at biosphere exhanges characterizing activity basalt_cs
for exc in basalt_cs.biosphere():
    print(exc)

Exchange: 8e-06 kilogram 'Particulate Matter, < 2.5 um' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.000112 kilogram 'Particulate Matter, > 10 um' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 4e-05 kilogram 'Particulate Matter, > 2.5 um and < 10um' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 1.68e-06 cubic meter 'Water' (cubic meter, None, ('air',)) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 6.72e-06 cubic meter 'Water' (cubic meter, None, ('water',)) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 1.0 kilogram 'Basalt' (kilogram, None, ('natural resource', 'in ground')) to 'basalt quarry operation (for crushed stone

In [69]:
#detail look to biosphere exchanges of interest
[exc for exc in basalt_cs.biosphere()][5].as_dict() #Basalt

{'flow': 'ac3a8914-35f0-4c34-a956-f26b3a053e4a',
 'type': 'biosphere',
 'name': 'Basalt',
 'classifications': {'CPC': []},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'water content': {'amount': 0.0, 'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'unit': 'kilogram',
 'amount': 1.0,
 'pedigree': {'reliability': 3,
  'completeness': 4,
  'temporal correlation': 5,
  'geographical correlation': 1,
  'further technological correlation': 3},
 'uncertainty type': 2,
 'loc': 0.0,
 'scale': 0.24899799195977465,
 'scale without pedigree': 0.1,
 'input': ('biosphere3', 'ac3a8914-35f0-4c34-a956-f26b3a053e4a'),
 'output'

In [70]:
# Iterate over biosphere exchanges in basalt_cs to change the amount of Basalt exchange to 0
for exc in basalt_cs.biosphere():
        if exc['name'] == 'Basalt':
            # Update the amount in the exchange
            exc['amount'] = 0
            exc.save()

In [71]:
#verify new exchange value for Basalt in basalt_cs
for exc in basalt_cs.biosphere():
    print(exc)

Exchange: 8e-06 kilogram 'Particulate Matter, < 2.5 um' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.000112 kilogram 'Particulate Matter, > 10 um' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 4e-05 kilogram 'Particulate Matter, > 2.5 um and < 10um' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 1.68e-06 cubic meter 'Water' (cubic meter, None, ('air',)) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 6.72e-06 cubic meter 'Water' (cubic meter, None, ('water',)) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0 kilogram 'Basalt' (kilogram, None, ('natural resource', 'in ground')) to 'basalt quarry operation (for crushed stone)'

In [75]:
#Look at technosphere exhanges characterizing activity basalt_cs
for exc in basalt_cs.technosphere():
    print(exc)

Exchange: 3.12e-06 kilogram 'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 8.0392e-05 kilogram 'blasting' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 3.744e-07 kilogram 'market for chemical, organic' (kilogram, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.043056000000000004 megajoule 'market for diesel, burned in building machine' (megajoule, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.0064584 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.000309244 megajoule 'market group for heat, district or industrial, other than natural gas' (megajoule, RER, None) to 'basalt quarry operatio

In [79]:
#detail look to technosphere exchange of interest
[exc for exc in basalt_cs.technosphere()][6].as_dict() #limestone quarry infrastructure

{'flow': '1e278342-7e3b-426d-b000-58eab8104573',
 'type': 'technosphere',
 'name': 'limestone quarry infrastructure',
 'classifications': {'CPC': ['53269: Other constructions for manufacturing']},
 'production volume': 0.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'}},
 'activity': 'e6a83983-3995-5ac3-9de9-d486f22896f9',
 'unit': 'unit',
 'comment': 'EcoSpold01Location=CH',
 'amount': 0,
 'pedigree': {'reliability': 5,
  'completeness': 4,
  'temporal correlation': 5,
  'geographical correlation': 1,
  'further technological correlation': 3},
 'uncertainty type': 2,
 'loc': -23.248994481254666,
 'scale': 0.6557438524302001,
 'scale without pedigree': 0.5830951894845301,
 'input': ('ecoinvent-391-cutoff', '8333c52e7f25a01b4e063b97466be74c'),
 'output': ('ecoinvent-391-cutoff', 'dff10c7f2b5f46928beb248896f9569b')}

In [78]:
# Iterate over technosphere exchanges in basalt_cs to change the amount of limestone quarry operation exchange to 0
for exc in basalt_cs.technosphere():
        if exc['name'] == 'limestone quarry infrastructure':
            # Update the amount in the exchange
            exc['amount'] = 0
            exc.save()

In [80]:
#verify new exchange value for limestone quarry operation in basalt_cs
for exc in basalt_cs.technosphere():
    print(exc)

Exchange: 3.12e-06 kilogram 'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 8.0392e-05 kilogram 'blasting' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 3.744e-07 kilogram 'market for chemical, organic' (kilogram, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.043056000000000004 megajoule 'market for diesel, burned in building machine' (megajoule, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.0064584 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.000309244 megajoule 'market group for heat, district or industrial, other than natural gas' (megajoule, RER, None) to 'basalt quarry operatio

In [72]:
#Look at technosphere exhanges characterizing activity basalt_cs
for exc in basalt_cs.technosphere():
    print(exc)

Exchange: 3e-06 kilogram 'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 7.73e-05 kilogram 'blasting' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 3.6e-07 kilogram 'market for chemical, organic' (kilogram, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.0414 megajoule 'market for diesel, burned in building machine' (megajoule, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.00621 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.00029735 megajoule 'market group for heat, district or industrial, other than natural gas' (megajoule, RER, None) to 'basalt quarry operation (for crushed stone)' (

In [73]:
# Iterate over technosphere exchanges in basalt_cs to multiply all technosphere exchanges time 1,04
for exc in basalt_cs.technosphere():
    # Update the amount in the exchange
    exc['amount'] = (exc['amount'])*1.04
    exc.save()
            

In [81]:
#verify new exchange technosphere values for basalt_cs
for exc in basalt_cs.technosphere():
    print(exc)

Exchange: 3.12e-06 kilogram 'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 8.0392e-05 kilogram 'blasting' (kilogram, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 3.744e-07 kilogram 'market for chemical, organic' (kilogram, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.043056000000000004 megajoule 'market for diesel, burned in building machine' (megajoule, GLO, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.0064584 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, RER, None) to 'basalt quarry operation (for crushed stone)' (kilogram, RER, None)>
Exchange: 0.000309244 megajoule 'market group for heat, district or industrial, other than natural gas' (megajoule, RER, None) to 'basalt quarry operatio

## Adding gravel production modified process

This processs is based on the following ecoinvent process:

- gravel production, crushed - RoW

The new process is modified to only include the following exchanges:

| Name                                               | Exchange type         | 
|----------------------------------------------------|-----------------------|
| gravel, crushed                                    | production            |
| building, hall, steel construction                 | technosphere          |
| conveyor belt                                      | technosphere          | 
| Gravel                                             | biosphere             | 
| gravel/sand quarry infrastructure                  | technosphere          | 
| industrial machine, heavy, unspecified             | technosphere          | 
| recultivation, limestone mine                      | technosphere          | 
| steel, low-alloyed, hot rolled                     | technosphere          | 
| synthetic rubber                                   | technosphere          | 


The name of the new process is set to:

- gravel production, crushed (for crushed stone) - RoW

In [82]:
#Search for gravel production, crushed in background database
EI_DB.search('gravel production, crushed')

['gravel production, crushed' (kilogram, BR, None),
 'gravel production, crushed' (kilogram, CH, None),
 'gravel production, crushed' (kilogram, RoW, None),
 'gravel production, crushed' (kilogram, CA-QC, None),
 'gravel production, crushed' (kilogram, IN, None),
 'market for gravel, crushed' (kilogram, IN, None),
 'market for gravel, crushed' (kilogram, RoW, None),
 'market for gravel, crushed' (kilogram, CA-QC, None),
 'market for gravel, crushed' (kilogram, CH, None),
 'gravel and sand quarry operation' (kilogram, CH, None),
 'gravel and sand quarry operation' (kilogram, RoW, None),
 'market for gravel, round' (kilogram, CH, None),
 'limestone production, crushed, for mill' (kilogram, RoW, None),
 'market for limestone, crushed, washed' (kilogram, RoW, None),
 'market for limestone, crushed, washed' (kilogram, CH, None),
 'market for limestone, crushed, for mill' (kilogram, RoW, None),
 'market for limestone, crushed, for mill' (kilogram, CH, None),
 'gravel and sand quarry operatio

In [83]:
#Copy existing gravel process
gravel = EI_DB.search('gravel production, crushed')[2]
gravel_cs = gravel.copy(name='gravel production, crushed (for crushed stone)') 

In [88]:
#Verify that the new gravel process was recorded
EI_DB.search('gravel production, crushed')

['gravel production, crushed' (kilogram, BR, None),
 'gravel production, crushed' (kilogram, CH, None),
 'gravel production, crushed' (kilogram, RoW, None),
 'gravel production, crushed' (kilogram, CA-QC, None),
 'gravel production, crushed' (kilogram, IN, None),
 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None),
 'market for gravel, crushed' (kilogram, IN, None),
 'market for gravel, crushed' (kilogram, RoW, None),
 'market for gravel, crushed' (kilogram, CA-QC, None),
 'market for gravel, crushed' (kilogram, CH, None),
 'gravel and sand quarry operation' (kilogram, CH, None),
 'gravel and sand quarry operation' (kilogram, RoW, None),
 'market for gravel, round' (kilogram, CH, None),
 'limestone production, crushed, for mill' (kilogram, RoW, None),
 'market for limestone, crushed, washed' (kilogram, RoW, None),
 'market for limestone, crushed, washed' (kilogram, CH, None),
 'market for limestone, crushed, for mill' (kilogram, RoW, None),
 'market for limestone, c

In [92]:
#Look at exhanges characterizing activity gravel_cs
for exc in gravel_cs.exchanges():
    print(exc)

Exchange: 1.0 kilogram 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 2.85e-06 square meter 'market for building, hall, steel construction' (square meter, GLO, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 9.51e-08 meter 'market for conveyor belt' (meter, GLO, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 0.0143 megajoule 'market for diesel, burned in building machine' (megajoule, GLO, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 6.484359817456601e-06 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, PY, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 9.613299192000142e-06 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, EC, None) to 'gravel production, crushed

In [93]:
#detail look to exchanges of interest
[exc for exc in gravel_cs.exchanges()][0].as_dict() #gravel, crushed

{'flow': '37eceabd-b33f-4757-a4b9-5c51dee1710d',
 'type': 'production',
 'name': 'gravel, crushed',
 'classifications': {'CPC': ['15320: Pebbles, gravel, broken or crushed stone, macadam; granules, chippings and powder of stone']},
 'production volume': 3684834500000.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0,
   'comment': 'SiO2, ecoinvent v2.2 report 7, part I',
   'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0,
   'comment': 'SiO2, ecoinvent v2.2 report 7, part I',
   'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'price': {'amount': 0.00446,
   'comment': 'World market prices are estimated to be equal to USA prices, which are given according to http://minerals.usgs.gov/minerals/pubs/commodity/sand_&_gravel_construction/myb1-2007-sandc.xls, table 1, downloaded 21 April 2009. NOTE: The crushing process

In [94]:
#create dictionary with exchanges to keep
gravel_cs_exc = [
    {"name": "gravel, crushed", "type": "production"},
    {"name": "building, hall, steel construction", "type": "technosphere"},
    {"name": "conveyor belt", "type": "technosphere"},
    {"name": "Gravel", "type": "biosphere"},
    {"name": "gravel/sand quarry infrastructure", "type": "technosphere"},
    {"name": "industrial machine, heavy, unspecified", "type": "technosphere"},
    {"name": "recultivation, limestone mine", "type": "technosphere"},
    {"name": "steel, low-alloyed, hot rolled", "type": "technosphere"},
    {"name": "synthetic rubber", "type": "technosphere"}
]

In [95]:
# Iterate over exchanges in gravel_cs to delete exchanges that are not included in gravel_cs_exc
for exc in gravel_cs.exchanges():
    found = False
    for item in gravel_cs_exc:
        if exc['name'] == item['name'] and exc['type'] == item['type']:
            found = True
            break
    if not found:
            exc.delete()
            exc.save()

In [96]:
#verify new exchanges for gravel_cs
for exc in gravel_cs.exchanges():
    print(exc)

Exchange: 1.0 kilogram 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 2.85e-06 square meter 'market for building, hall, steel construction' (square meter, GLO, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 9.51e-08 meter 'market for conveyor belt' (meter, GLO, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 4.75e-11 unit 'market for gravel/sand quarry infrastructure' (unit, GLO, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 9.51e-05 kilogram 'market for industrial machine, heavy, unspecified' (kilogram, RoW, None) to 'gravel production, crushed (for crushed stone)' (kilogram, RoW, None)>
Exchange: 1.27e-06 square meter 'market for recultivation, limestone mine' (square meter, GLO, None) to 'gravel production, crushed (for crushed stone)' (kilogram,

## Adding cellullose fibre production modified process

This processs is based on the following ecoinvent process:

- cellulose fibre production - RoW

The new process is modified to exclude the *borax, anhydrous, powder* and *boric acid, anhydrous, powder* technosphere exchanges.

The name of the new process is set to:

- cellulose fibre production (without borax and boric acid) - RoW

In [106]:
#Search for cellulose fibre production in background database
EI_DB.search('cellulose fibre production')

['cellulose fibre production' (kilogram, CH, None),
 'cellulose fibre production' (kilogram, RoW, None)]

In [107]:
#Copy existing cellulose fibre process
cellulose = EI_DB.search('cellulose fibre production')[1]
cellulose_mod = cellulose.copy(name='cellulose fibre production (without borax and boric acid)') 

In [108]:
#Verify that the new cellulose fibre process was recorded
EI_DB.search('cellulose fibre production')

['cellulose fibre production' (kilogram, CH, None),
 'cellulose fibre production' (kilogram, RoW, None),
 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)]

In [109]:
#Look at technosphere exhanges characterizing activity cellulose_mod
for exc in cellulose_mod.technosphere():
    print(exc)

Exchange: 0.04 kilogram 'market for aluminium hydroxide' (kilogram, GLO, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.04 kilogram 'market for boric acid, anhydrous, powder' (kilogram, GLO, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 4.24e-06 square meter 'market for building, hall, steel construction' (square meter, GLO, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.0011939174733314618 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, AU, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.07209155511042603 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, RAS, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.00785732043731028 kilowatt hour 'market group for ele

In [110]:
# Iterate over exchanges in cellulose_mod to delete borax, anhydrous, powder and boric acid, anhydrous, powder technosphere exchanges
for exc in cellulose_mod.technosphere():
    if exc['name'] == 'borax, anhydrous, powder':
        exc.delete()
        exc.save()
    elif exc['name'] == 'boric acid, anhydrous, powder':
        exc.delete()
        exc.save()

In [111]:
#verify new technosphere exchanges for cellulose_mod
for exc in cellulose_mod.technosphere():
    print(exc)

Exchange: 0.04 kilogram 'market for aluminium hydroxide' (kilogram, GLO, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 4.24e-06 square meter 'market for building, hall, steel construction' (square meter, GLO, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.0011939174733314618 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, AU, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.07209155511042603 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, RAS, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.00785732043731028 kilowatt hour 'market group for electricity, medium voltage' (kilowatt hour, RLA, None) to 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)>
Exchange: 0.017154767770878203 kilow